In [202]:
import glob
import os
import pandas as pd

define constants

In [218]:
keys = ["top","tok","mess","rush","sil","s_m","s_n","er","EOS",]
column_names = ["Mwfsending", "Mwfreceiving", "Mwfreturning", "Mcirctime", "top","tok","mess","rush","sil","s_m","s_n","er"]
path_to_write_data = "D:\\Git\\TokenRing\\research\\"
path_to_data = path_to_write_data+"data\\"
filename = "results.txt"

define functions

In [204]:
def createColumnNames():
    result = ""
    for column_name in column_names:
        result = result + column_name + " "
    return result

In [206]:
def processFilename(filename):
    result = ""
    for x in range(0, len(keys)-1):
        result = result + filename[filename.find(keys[x])+len(keys[x]): filename.find(keys[x+1])]+" "
    return result

In [207]:
def appendStr(appendix):
    if os.path.exists(path_to_write_data+filename):
        append_write = 'a' # append if already exists
    else:
        append_write = 'w' # make a new file if not

    file_with_results = open(path_to_write_data+filename, append_write)
    if append_write == "w":
            file_with_results.write(createColumnNames()+"\n")
    file_with_results.write(appendix+"\n")
    file_with_results.close()

In [209]:
def collectResults(filename):
    names = ['Created', 'Sent', 'Received', 'Returned']
    data = pd.read_table(filename, sep=' ', names=names)
    data = data.drop(data.index[0])
    data = data.apply(pd.to_numeric, errors='coerce')
    wait_for_sending = data["Sent"] - data["Created"]
    wait_for_receiving = data["Received"] - data["Created"]
    circulated_before_receiving = data["Received"] - data["Sent"]
    time = data["Created"] - min(data["Created"])
    wait_for_returning = data["Returned"] - data["Created"]
    circulated_before_return = data["Returned"] - data["Sent"]
    not_returned = data.loc[data["Returned"].values==0].index
    circulated_before_return_good = circulated_before_return.drop(not_returned)
    wait_for_returning_good = wait_for_returning.drop(not_returned)
    time_good = time.drop(not_returned)
    result = str(wait_for_sending.median())
    result += " "
    result += str(wait_for_receiving.median())
    result += " "
    result += str(wait_for_returning_good.median())
    result += " "
    result += str(circulated_before_return_good.median())
    result += " "
    return result

In [219]:
def takeDataFromFile(elem):
    name = elem.replace("D:\\Git\\TokenRing\\", "")
    result = collectResults(elem)+processFilename(name)
    print result
    return result

create dataset with all measurements

In [220]:
t = glob.glob(path_to_data+"*.txt")

In [221]:
for elem in t:
    appendStr(takeDataFromFile(elem))

53019996.0 53820526.5 54594061.0 1668888.0 10 3 1000 true true 10 10 true 
